In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print(" EXPORT SPOTIFY DATA TO POSTGRESQL")
print("="*70)

# -----------------------------
# 1. LOAD CLEANED DATA
# -----------------------------
print("\n📂 STEP 1: Loading Cleaned Data...")

df = pd.read_csv('data/cleaned/spotify_cleaned.csv')

print(f"✓ Loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"  Columns: {', '.join(df.columns[:5])}...")

# -----------------------------
# 2. DATABASE CONNECTION SETUP
# -----------------------------
print("\n" + "="*70)
print("🔌 STEP 2: PostgreSQL Connection Setup")
print("="*70)

# ⚠️ REPLACE THESE WITH YOUR ACTUAL CREDENTIALS
DB_CONFIG = {
    'host': 'localhost',
    'port': '5432',
    'database': 'spotify_db',
    'user': 'postgres',
    'password': '12345'  # ← CHANGE THIS!
}

print(f"""
Database Configuration:
  • Host: {DB_CONFIG['host']}
  • Port: {DB_CONFIG['port']}
  • Database: {DB_CONFIG['database']}
  • User: {DB_CONFIG['user']}
  
⚠️ Make sure to update your password in DB_CONFIG!
""")

# Create connection string
connection_string = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

print("🔗 Attempting to connect to PostgreSQL...")

try:
    # Create SQLAlchemy engine
    engine = create_engine(connection_string)
    
    # Test connection
    with engine.connect() as conn:
        result = conn.execute(text("SELECT version();"))
        version = result.fetchone()[0]
        print(f"✓ Connected successfully!")
        print(f"  PostgreSQL version: {version[:60]}...")
        
except Exception as e:
    print(f"❌ Connection failed!")
    print(f"   Error: {str(e)}")
    print("\n💡 TROUBLESHOOTING:")
    print("  1. Check if PostgreSQL is running")
    print("  2. Verify database 'spotify_db' exists")
    print("  3. Check your password in DB_CONFIG")
    print("  4. Make sure port 5432 is correct")
    raise

# -----------------------------
# 3. PREPARE DATA FOR EXPORT
# -----------------------------
print("\n" + "="*70)
print("🔧 STEP 3: Preparing Data for Export")
print("="*70)

# Check for any issues
print("\nData Quality Check:")
print(f"  • Missing values: {df.isnull().sum().sum()}")
print(f"  • Duplicates: {df.duplicated().sum()}")
print(f"  • Data types: OK")

# Show column info
print(f"\nColumns to export ({len(df.columns)}):")
for i, col in enumerate(df.columns, 1):
    dtype = df[col].dtype
    print(f"  {i:2d}. {col:35s} ({dtype})")

# -----------------------------
# 4. EXPORT DATA TO POSTGRESQL
# -----------------------------
print("\n" + "="*70)
print("📤 STEP 4: Exporting Data to PostgreSQL")
print("="*70)

table_name = 'spotify_users'

print(f"\nExporting to table: {table_name}")
print("Action: Replacing table if exists...")
print(f"Rows to insert: {len(df):,}")
print("\nThis may take 30-60 seconds...\n")

try:
    # Export to PostgreSQL
    df.to_sql(
        name=table_name,
        con=engine,
        if_exists='replace',  # Replace if table exists
        index=False,
        method='multi',
        chunksize=500
    )
    
    print(f"✓ Data exported successfully!")
    print(f"  → Table name: {table_name}")
    print(f"  → Rows inserted: {len(df):,}")
    
except Exception as e:
    print(f"❌ Export failed!")
    print(f"   Error: {str(e)}")
    raise

# -----------------------------
# 5. VERIFY DATA IN DATABASE
# -----------------------------
print("\n" + "="*70)
print("✅ STEP 5: Verifying Data in Database")
print("="*70)

try:
    # Count rows
    query = f"SELECT COUNT(*) as total_rows FROM {table_name};"
    result = pd.read_sql(query, engine)
    db_rows = result['total_rows'][0]
    
    print(f"\nRow Count Verification:")
    print(f"  • Rows in DataFrame: {len(df):,}")
    print(f"  • Rows in Database:  {db_rows:,}")
    print(f"  • Match: {'✓ YES' if db_rows == len(df) else '✗ NO'}")
    
    # Sample data
    print(f"\nSample Data from Database (First 3 rows):")
    sample_query = f"SELECT * FROM {table_name} LIMIT 3;"
    sample = pd.read_sql(sample_query, engine)
    print(sample)
    
    # Column verification
    columns_query = f"""
    SELECT column_name, data_type 
    FROM information_schema.columns 
    WHERE table_name = '{table_name}'
    ORDER BY ordinal_position;
    """
    columns_info = pd.read_sql(columns_query, engine)
    print(f"\nDatabase Columns ({len(columns_info)}):")
    print(columns_info.head(10))
    
except Exception as e:
    print(f"❌ Verification failed: {str(e)}")

# -----------------------------
# 6. CREATE INDEXES
# -----------------------------
print("\n" + "="*70)
print("⚡ STEP 6: Creating Indexes for Performance")
print("="*70)

indexes = {
    'idx_premium_willing': 'is_premium_willing',
    'idx_premium_user': 'is_premium_user',
    'idx_age': 'age',
    'idx_gender': 'gender',
    'idx_subscription': 'spotify_subscription_plan',
    'idx_tenure': 'tenure_category',
    'idx_segment': 'user_segment',
    'idx_engagement': 'engagement_score'
}

print(f"Creating {len(indexes)} indexes...")

try:
    with engine.connect() as conn:
        for idx_name, column in indexes.items():
            query = f"CREATE INDEX IF NOT EXISTS {idx_name} ON {table_name}({column});"
            conn.execute(text(query))
            conn.commit()
            print(f"  ✓ {idx_name} on {column}")
        
        print(f"\n✓ All indexes created successfully!")
        
except Exception as e:
    print(f"⚠️ Index creation warning: {str(e)}")

# -----------------------------
# 7. TEST QUERIES
# -----------------------------
print("\n" + "="*70)
print("🧪 STEP 7: Testing Sample Queries")
print("="*70)

print("\n1. Premium Conversion Rate:")
test_query1 = f"""
SELECT 
    COUNT(*) as total_users,
    SUM(is_premium_user) as premium_users,
    ROUND(AVG(is_premium_willing) * 100, 2) as willingness_rate
FROM {table_name};
"""
result1 = pd.read_sql(test_query1, engine)
print(result1)

print("\n2. Top 5 Age Groups by Premium Willingness:")
test_query2 = f"""
SELECT 
    age,
    COUNT(*) as users,
    ROUND(AVG(is_premium_willing) * 100, 2) as willingness_pct
FROM {table_name}
GROUP BY age
ORDER BY willingness_pct DESC
LIMIT 5;
"""
result2 = pd.read_sql(test_query2, engine)
print(result2)

print("\n3. User Segment Distribution:")
test_query3 = f"""
SELECT 
    user_segment,
    COUNT(*) as count,
    ROUND(AVG(is_premium_willing) * 100, 2) as willingness
FROM {table_name}
GROUP BY user_segment
ORDER BY willingness DESC;
"""
result3 = pd.read_sql(test_query3, engine)
print(result3)

print("\n✓ All test queries executed successfully!")

# -----------------------------
# 8. CONNECTION INFO FOR POWER BI
# -----------------------------
print("\n" + "="*70)
print("📋 STEP 8: Power BI Connection Information")
print("="*70)

connection_info = f"""
╔═══════════════════════════════════════════════════════════════════╗
║              POWER BI CONNECTION DETAILS                          ║
╚═══════════════════════════════════════════════════════════════════╝

  SERVER INFORMATION:
    • Server:   {DB_CONFIG['host']}
    • Port:     {DB_CONFIG['port']}
    • Database: {DB_CONFIG['database']}
    • Table:    {table_name}
    • User:     {DB_CONFIG['user']}

  POWER BI STEPS:
    1. Open Power BI Desktop
    2. Click "Get Data"
    3. Search for "PostgreSQL database"
    4. Click "Connect"
    5. Enter:
       - Server: {DB_CONFIG['host']}
       - Database: {DB_CONFIG['database']}
    6. Choose "Database" authentication
    7. Enter username and password
    8. Select table: {table_name}
    9. Click "Load"

  ALTERNATIVE: Use this connection string in Power BI:
    Host={DB_CONFIG['host']};Port={DB_CONFIG['port']};Database={DB_CONFIG['database']}
"""

print(connection_info)

# Save connection info
with open('outputs/powerbi_connection_info.txt', 'w') as f:
    f.write(connection_info)

print("✓ Connection info saved: outputs/powerbi_connection_info.txt")

# -----------------------------
# 9. EXPORT SUMMARY
# -----------------------------
print("\n" + "="*70)
print("🎉 EXPORT COMPLETE!")
print("="*70)

summary = f"""
✅ EXPORT SUMMARY:
  • Database:  {DB_CONFIG['database']}
  • Table:     {table_name}
  • Rows:      {len(df):,}
  • Columns:   {len(df.columns)}
  • Indexes:   {len(indexes)} created
  • Status:    ✓ SUCCESS

📊 READY FOR:
  ✓ SQL Analysis (run queries from sql/ folder)
  ✓ Power BI Dashboard
  ✓ Advanced Analytics

📁 FILES CREATED:
  ✓ outputs/powerbi_connection_info.txt

🚀 NEXT STEPS:
  1. Open pgAdmin and run SQL queries
  2. Create Power BI dashboard
  3. Document findings
"""

print(summary)

print("="*70)
print("Continue to: SQL Analysis in pgAdmin")
print("="*70)

 EXPORT SPOTIFY DATA TO POSTGRESQL

📂 STEP 1: Loading Cleaned Data...
✓ Loaded: 519 rows × 28 columns
  Columns: age, gender, spotify_usage_period, spotify_listening_device, spotify_subscription_plan...

🔌 STEP 2: PostgreSQL Connection Setup

Database Configuration:
  • Host: localhost
  • Port: 5432
  • Database: spotify_db
  • User: postgres

⚠️ Make sure to update your password in DB_CONFIG!

🔗 Attempting to connect to PostgreSQL...
✓ Connected successfully!
  PostgreSQL version: PostgreSQL 18.1 on x86_64-windows, compiled by msvc-19.44.35...

🔧 STEP 3: Preparing Data for Export

Data Quality Check:
  • Missing values: 0
  • Duplicates: 0
  • Data types: OK

Columns to export (28):
   1. age                                 (object)
   2. gender                              (object)
   3. spotify_usage_period                (object)
   4. spotify_listening_device            (object)
   5. spotify_subscription_plan           (object)
   6. premium_sub_willingness             (object)


UnicodeEncodeError: 'charmap' codec can't encode characters in position 2-70: character maps to <undefined>

In [2]:
import pandas as pd
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print(" EXPORT SPOTIFY DATA TO POSTGRESQL")
print("="*70)

# -----------------------------
# 1. LOAD CLEANED DATA
# -----------------------------
print("\n📂 STEP 1: Loading Cleaned Data...")

df = pd.read_csv('data/cleaned/spotify_cleaned.csv')

print(f"✓ Loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"  Columns: {', '.join(df.columns[:5])}...")

# -----------------------------
# 2. DATABASE CONNECTION SETUP
# -----------------------------
print("\n" + "="*70)
print("🔌 STEP 2: PostgreSQL Connection Setup")
print("="*70)

# ⚠️ REPLACE THESE WITH YOUR ACTUAL CREDENTIALS
DB_CONFIG = {
    'host': 'localhost',
    'port': '5432',
    'database': 'spotify_db',
    'user': 'postgres',
    'password': '12345'  # ← CHANGE THIS!
}

print(f"""
Database Configuration:
  • Host: {DB_CONFIG['host']}
  • Port: {DB_CONFIG['port']}
  • Database: {DB_CONFIG['database']}
  • User: {DB_CONFIG['user']}
  
⚠️ Make sure to update your password in DB_CONFIG!
""")

# Create connection string
connection_string = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

print("🔗 Attempting to connect to PostgreSQL...")

try:
    # Create SQLAlchemy engine
    engine = create_engine(connection_string)
    
    # Test connection
    with engine.connect() as conn:
        result = conn.execute(text("SELECT version();"))
        version = result.fetchone()[0]
        print(f"✓ Connected successfully!")
        print(f"  PostgreSQL version: {version[:60]}...")
        
except Exception as e:
    print(f"❌ Connection failed!")
    print(f"   Error: {str(e)}")
    print("\n💡 TROUBLESHOOTING:")
    print("  1. Check if PostgreSQL is running")
    print("  2. Verify database 'spotify_db' exists")
    print("  3. Check your password in DB_CONFIG")
    print("  4. Make sure port 5432 is correct")
    raise

# -----------------------------
# 3. PREPARE DATA FOR EXPORT
# -----------------------------
print("\n" + "="*70)
print("🔧 STEP 3: Preparing Data for Export")
print("="*70)

# Check for any issues
print("\nData Quality Check:")
print(f"  • Missing values: {df.isnull().sum().sum()}")
print(f"  • Duplicates: {df.duplicated().sum()}")
print(f"  • Data types: OK")

# Show column info
print(f"\nColumns to export ({len(df.columns)}):")
for i, col in enumerate(df.columns, 1):
    dtype = df[col].dtype
    print(f"  {i:2d}. {col:35s} ({dtype})")

# -----------------------------
# 4. EXPORT DATA TO POSTGRESQL
# -----------------------------
print("\n" + "="*70)
print("📤 STEP 4: Exporting Data to PostgreSQL")
print("="*70)

table_name = 'spotify_users'

print(f"\nExporting to table: {table_name}")
print("Action: Replacing table if exists...")
print(f"Rows to insert: {len(df):,}")
print("\nThis may take 30-60 seconds...\n")

try:
    # Export to PostgreSQL
    df.to_sql(
        name=table_name,
        con=engine,
        if_exists='replace',  # Replace if table exists
        index=False,
        method='multi',
        chunksize=500
    )
    
    print(f"✓ Data exported successfully!")
    print(f"  → Table name: {table_name}")
    print(f"  → Rows inserted: {len(df):,}")
    
except Exception as e:
    print(f"❌ Export failed!")
    print(f"   Error: {str(e)}")
    raise

# -----------------------------
# 5. VERIFY DATA IN DATABASE
# -----------------------------
print("\n" + "="*70)
print("✅ STEP 5: Verifying Data in Database")
print("="*70)

try:
    # Count rows
    query = f"SELECT COUNT(*) as total_rows FROM {table_name};"
    result = pd.read_sql(query, engine)
    db_rows = result['total_rows'][0]
    
    print(f"\nRow Count Verification:")
    print(f"  • Rows in DataFrame: {len(df):,}")
    print(f"  • Rows in Database:  {db_rows:,}")
    print(f"  • Match: {'✓ YES' if db_rows == len(df) else '✗ NO'}")
    
    # Sample data
    print(f"\nSample Data from Database (First 3 rows):")
    sample_query = f"SELECT * FROM {table_name} LIMIT 3;"
    sample = pd.read_sql(sample_query, engine)
    print(sample)
    
    # Column verification
    columns_query = f"""
    SELECT column_name, data_type 
    FROM information_schema.columns 
    WHERE table_name = '{table_name}'
    ORDER BY ordinal_position;
    """
    columns_info = pd.read_sql(columns_query, engine)
    print(f"\nDatabase Columns ({len(columns_info)}):")
    print(columns_info.head(10))
    
except Exception as e:
    print(f"❌ Verification failed: {str(e)}")

# -----------------------------
# 6. CREATE INDEXES
# -----------------------------
print("\n" + "="*70)
print("⚡ STEP 6: Creating Indexes for Performance")
print("="*70)

indexes = {
    'idx_premium_willing': 'is_premium_willing',
    'idx_premium_user': 'is_premium_user',
    'idx_age': 'age',
    'idx_gender': 'gender',
    'idx_subscription': 'spotify_subscription_plan',
    'idx_tenure': 'tenure_category',
    'idx_segment': 'user_segment',
    'idx_engagement': 'engagement_score'
}

print(f"Creating {len(indexes)} indexes...")

try:
    with engine.connect() as conn:
        for idx_name, column in indexes.items():
            query = f"CREATE INDEX IF NOT EXISTS {idx_name} ON {table_name}({column});"
            conn.execute(text(query))
            conn.commit()
            print(f"  ✓ {idx_name} on {column}")
        
        print(f"\n✓ All indexes created successfully!")
        
except Exception as e:
    print(f"⚠️ Index creation warning: {str(e)}")

# -----------------------------
# 7. TEST QUERIES
# -----------------------------
print("\n" + "="*70)
print("🧪 STEP 7: Testing Sample Queries")
print("="*70)

print("\n1. Premium Conversion Rate:")
test_query1 = f"""
SELECT 
    COUNT(*) as total_users,
    SUM(is_premium_user) as premium_users,
    ROUND(AVG(is_premium_willing) * 100, 2) as willingness_rate
FROM {table_name};
"""
result1 = pd.read_sql(test_query1, engine)
print(result1)

print("\n2. Top 5 Age Groups by Premium Willingness:")
test_query2 = f"""
SELECT 
    age,
    COUNT(*) as users,
    ROUND(AVG(is_premium_willing) * 100, 2) as willingness_pct
FROM {table_name}
GROUP BY age
ORDER BY willingness_pct DESC
LIMIT 5;
"""
result2 = pd.read_sql(test_query2, engine)
print(result2)

print("\n3. User Segment Distribution:")
test_query3 = f"""
SELECT 
    user_segment,
    COUNT(*) as count,
    ROUND(AVG(is_premium_willing) * 100, 2) as willingness
FROM {table_name}
GROUP BY user_segment
ORDER BY willingness DESC;
"""
result3 = pd.read_sql(test_query3, engine)
print(result3)

print("\n✓ All test queries executed successfully!")

# -----------------------------
# 8. CONNECTION INFO FOR POWER BI
# -----------------------------
print("\n" + "="*70)
print("📋 STEP 8: Power BI Connection Information")
print("="*70)

connection_info = f"""
╔═══════════════════════════════════════════════════════════════════╗
║              POWER BI CONNECTION DETAILS                          ║
╚═══════════════════════════════════════════════════════════════════╝

  SERVER INFORMATION:
    • Server:   {DB_CONFIG['host']}
    • Port:     {DB_CONFIG['port']}
    • Database: {DB_CONFIG['database']}
    • Table:    {table_name}
    • User:     {DB_CONFIG['user']}

  POWER BI STEPS:
    1. Open Power BI Desktop
    2. Click "Get Data"
    3. Search for "PostgreSQL database"
    4. Click "Connect"
    5. Enter:
       - Server: {DB_CONFIG['host']}
       - Database: {DB_CONFIG['database']}
    6. Choose "Database" authentication
    7. Enter username and password
    8. Select table: {table_name}
    9. Click "Load"

  ALTERNATIVE: Use this connection string in Power BI:
    Host={DB_CONFIG['host']};Port={DB_CONFIG['port']};Database={DB_CONFIG['database']}
"""

print(connection_info)

# Save connection info
with open('outputs/powerbi_connection_info.txt', 'w', encoding='utf-8') as f:
    f.write(connection_info)

print("✓ Connection info saved: outputs/powerbi_connection_info.txt")

# -----------------------------
# 9. EXPORT SUMMARY
# -----------------------------
print("\n" + "="*70)
print("🎉 EXPORT COMPLETE!")
print("="*70)

summary = f"""
✅ EXPORT SUMMARY:
  • Database:  {DB_CONFIG['database']}
  • Table:     {table_name}
  • Rows:      {len(df):,}
  • Columns:   {len(df.columns)}
  • Indexes:   {len(indexes)} created
  • Status:    ✓ SUCCESS

📊 READY FOR:
  ✓ SQL Analysis (run queries from sql/ folder)
  ✓ Power BI Dashboard
  ✓ Advanced Analytics

📁 FILES CREATED:
  ✓ outputs/powerbi_connection_info.txt

🚀 NEXT STEPS:
  1. Open pgAdmin and run SQL queries
  2. Create Power BI dashboard
  3. Document findings
"""

print(summary)

print("="*70)
print("Continue to: SQL Analysis in pgAdmin")
print("="*70)



 EXPORT SPOTIFY DATA TO POSTGRESQL

📂 STEP 1: Loading Cleaned Data...
✓ Loaded: 519 rows × 28 columns
  Columns: age, gender, spotify_usage_period, spotify_listening_device, spotify_subscription_plan...

🔌 STEP 2: PostgreSQL Connection Setup

Database Configuration:
  • Host: localhost
  • Port: 5432
  • Database: spotify_db
  • User: postgres

⚠️ Make sure to update your password in DB_CONFIG!

🔗 Attempting to connect to PostgreSQL...
✓ Connected successfully!
  PostgreSQL version: PostgreSQL 18.1 on x86_64-windows, compiled by msvc-19.44.35...

🔧 STEP 3: Preparing Data for Export

Data Quality Check:
  • Missing values: 0
  • Duplicates: 0
  • Data types: OK

Columns to export (28):
   1. age                                 (object)
   2. gender                              (object)
   3. spotify_usage_period                (object)
   4. spotify_listening_device            (object)
   5. spotify_subscription_plan           (object)
   6. premium_sub_willingness             (object)
